In [51]:
import pandas as pd
import numpy as np
from numpy import *

In [2]:
p = pd.read_csv('data/large_files/final_data_with_text_stat.csv')

In [3]:
c = list(p.columns)[1:73] + list(p.columns)[173:185]

In [4]:
p = p[c]

In [24]:
train_age = pd.read_csv('data/train_age.csv')
test_user = pd.read_csv('data/test_user.csv')
n_train = len(train_age['age'])

In [6]:
train_data = p[:n_train]
test_data = p[n_train:]

In [7]:
p = 0
del p 

In [8]:
train_vecs = pd.read_csv('data/trainDataVecs400.csv', header = None)
test_vecs = pd.read_csv('data/testDataVecs400.csv', header = None)

In [11]:
test_data = test_data.reset_index(drop=True)

In [13]:
train_data = pd.concat([train_data, train_vecs], axis=1)

In [14]:
test_data = pd.concat([test_data, test_vecs], axis=1)

In [15]:
train_vec = 0
test_vec = 0
del train_vecs
del test_vecs
import gc
gc.collect()

7

In [16]:
train_data = pd.merge(train_data, train_age, on='post.id')

In [17]:
test_data = pd.merge(test_data, test_user, on='post.id')

In [19]:
c = list(train_data.columns)

In [22]:
c.remove('Unnamed: 0')

In [23]:
c.remove('post.id')

In [25]:
c.remove('age')

In [26]:
train_data = train_data[c]
test_data = test_data[c]

In [ ]:
c.remove('user.id')

In [29]:
train_data_g_id = train_data.groupby('user.id', as_index=False)[c].mean()
test_data_g_id = test_data.groupby('user.id', as_index=False)[c].mean()

In [39]:
train_age = train_age.groupby('user.id', as_index=False)['age'].mean()

In [51]:
train_cols = list(train_data.columns)
train_cols.remove('user.id')

In [60]:
# Initializing and fitting logistic regression model
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.svm import SVR

In [344]:
def mse_cv(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    model.fit(X_train, y_train)
    true = y_test
    preds = model.predict(X_test)
    print(preds)
    rmse= np.mean(np.square(np.absolute(preds - true)))
    return(rmse)

def get_preds_models(model1, model2, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    model1.fit(X_train, y_train)
    model2.fit(X_train, y_train)
    true = y_test
    model1_preds = model1.predict(X_test)
    model2_preds = model2.predict(X_test)
    return(model1_preds, model2_preds, true)

def get_preds_models_krr(model1, model2, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(X_train)
    model1.fit(train_scaled, y_train)
    model2.fit(X_train, y_train)
    true = y_test
    test_scaled = scaler.fit_transform(X_test)
    model1_preds = model1.predict(test_scaled)
    model2_preds = model2.predict(X_test)
    return(model1_preds, model2_preds, true)

def get_preds_models_krr_u(model1, model2, model3, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=4)
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    scaler = MinMaxScaler()
    
    train_scaled = scaler.fit_transform(X_train[keep])
    
    model1.fit(train_scaled, y_train)
    model2.fit(X_train, y_train)
    model3.fit(X_train[keep], y_train)
    
    true = y_test
    
    test_scaled = scaler.fit_transform(X_test[keep])
    
    model1_preds = model1.predict(test_scaled)
    model2_preds = model2.predict(X_test)
    model3_preds = model3.predict(X_test[keep])
    
    return(model1_preds, model2_preds, model3_preds, true)

In [54]:
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data_g_id[train_cols])
krr = KernelRidge(alpha=0.1, kernel='polynomial')
mse_cv(krr, train_scaled, train_age['age'])

[17.11203391 24.37812075 18.82328682 ... 27.30786743 25.09624133
 26.72932937]


29.02319219070675

In [58]:
xgbm = xgb.XGBRegressor(n_estimators=360, max_depth=4, learning_rate=0.1)
mse_cv(xgbm, train_data_g_id, train_age['age'])

[16.051346 24.487913 16.0046   ... 27.13723  19.929619 26.749685]


29.153238112985004

In [53]:
train_data_g.fillna(0)

,length_text,Positive,Negative,Neutral,Compound,gender_female,gender_male,gender_nan,sign_Aquarius,sign_Aries,...,391,392,393,394,395,396,397,398,399,user.id
0,157,0.000000,0.000000,1.000000,0.000000,0,1,0,0,0,...,-0.008306,0.026309,-0.007775,0.022020,0.041089,-0.013629,0.009564,0.000502,0.049659,11869
1,181,0.000000,0.000000,1.000000,0.000000,0,1,0,0,0,...,-0.041140,-0.014168,-0.019029,-0.012808,0.045717,0.000823,-0.020342,-0.012189,0.041024,11869
2,25467,0.081101,0.070965,0.847934,0.003812,0,1,0,0,0,...,0.012934,-0.005714,0.008087,0.005792,0.005074,-0.017524,-0.000149,0.011850,0.014451,11869
3,43,0.000000,0.000000,0.666667,0.000000,0,1,0,0,0,...,0.054530,-0.039987,0.045703,0.040441,0.089313,-0.030253,-0.010423,-0.018662,-0.000754,11869
4,402,0.430500,0.000000,0.569500,0.393100,0,1,0,1,0,...,-0.013637,0.015526,0.004629,0.028332,0.039243,-0.012475,-0.015321,-0.009130,0.029293,16332
5,3695,0.065538,0.029974,0.904487,0.088338,0,1,0,1,0,...,-0.000577,0.006512,-0.005388,0.007967,0.010873,-0.023791,0.003578,-0.003207,0.025500,16332
6,1105,0.106000,0.102286,0.791714,0.123314,0,1,0,1,0,...,-0.001003,0.020358,0.019004,0.008829,0.008447,-0.003021,-0.009319,0.004589,-0.002275,16332
7,2281,0.056043,0.077870,0.866087,-0.042148,0,1,0,1,0,...,0.006220,0.016275,-0.012761,0.005193,0.007150,-0.022283,-0.001399,-0.001499,0.011531,16332
8,2289,0.099250,0.088000,0.812833,0.030350,0,1,0,1,0,...,0.017873,0.023173,-0.003769,-0.002369,0.013336,-0.021988,-0.010795,-0.001147,0.014154,16332
9,957,0.072750,0.053125,0.874250,-0.024063,0,1,0,1,0,...,-0.001529,0.023210,-0.002369,0.016123,0.015694,-0.025140,-0.010374,0.004123,0.022759,16332


In [57]:
train_data_g_id.to_csv('train_data_vecs400_tfidf0_grouped.csv', encoding='utf-8')
test_data_g_id.to_csv('test_data_vecs400_tfidf0_grouped.csv', encoding='utf-8')

In [15]:
train_tfidf = pd.read_csv('train_data_vecs_tfidf.csv')

In [16]:
test_tfidf = pd.read_csv('test_data_vecs_tfidf.csv')

In [22]:
tfidf_c = list(train_tfidf.columns)[184:]

In [23]:
train_tfidf = train_tfidf[tfidf_c]
test_tfidf = test_tfidf[tfidf_c]

In [25]:
train_tfidf = pd.merge(train_tfidf, train_age, on='post.id')
test_tfidf = pd.merge(test_tfidf, test_user, on='post.id')

In [28]:
tfidf_c = list(train_tfidf.columns)

In [29]:
tfidf_c

['post.id',
 'actually',
 'also',
 'always',
 'another',
 'anything',
 'around',
 'away',
 'back',
 'bad',
 'best',
 'better',
 'blog',
 'came',
 'come',
 'could',
 'day',
 'days',
 'even',
 'ever',
 'every',
 'everyone',
 'feel',
 'find',
 'first',
 'friends',
 'fun',
 'get',
 'getting',
 'go',
 'god',
 'going',
 'good',
 'got',
 'great',
 'home',
 'house',
 'im',
 'know',
 'last',
 'let',
 'life',
 'like',
 'little',
 'long',
 'look',
 'lot',
 'love',
 'made',
 'make',
 'man',
 'many',
 'may',
 'maybe',
 'much',
 'na',
 'need',
 'never',
 'new',
 'next',
 'night',
 'nothing',
 'oh',
 'old',
 'one',
 'people',
 'pretty',
 'really',
 'right',
 'said',
 'say',
 'school',
 'see',
 'since',
 'someone',
 'something',
 'still',
 'sure',
 'take',
 'tell',
 'thing',
 'things',
 'think',
 'though',
 'thought',
 'time',
 'today',
 'two',
 'u',
 'urllink',
 'us',
 'want',
 'way',
 'week',
 'well',
 'went',
 'work',
 'world',
 'would',
 'year',
 'years',
 'Unnamed: 0',
 'age',
 'user.id']

In [30]:
tfidf_c.remove('Unnamed: 0')
tfidf_c.remove('age')
tfidf_c.remove('post.id')

In [31]:
train_tfidf_g_id = train_tfidf.groupby('user.id', as_index=False)[tfidf_c].mean()
test_tfidf_g_id = test_tfidf.groupby('user.id', as_index=False)[tfidf_c].mean()

In [34]:
train_tfidf = 0
test_tfidf = 0
del train_tfidf
del test_tfidf

In [35]:
train_data = pd.read_csv('train_data_vecs400_tfidf0_grouped.csv', encoding='utf-8')
test_data = pd.read_csv('test_data_vecs400_tfidf0_grouped.csv', encoding='utf-8')

In [41]:
train_data = pd.merge(train_data, train_tfidf_g_id, on='user.id')

In [42]:
test_data = pd.merge(test_data, test_tfidf_g_id, on='user.id')

In [44]:
train_age = train_age.groupby('user.id', as_index=False)['age'].mean()

In [46]:
train_data = pd.merge(train_data, train_age, on='user.id')

In [57]:
train_cols = list(train_data.columns)[1:]

In [59]:
train_cols.remove('user.id')
train_cols.remove('age')

In [62]:
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data[train_cols])
krr = KernelRidge(alpha=0.1, kernel='polynomial')
mse_cv(krr, train_scaled, train_data['age'])

[16.50426265 23.56579283 18.8500901  ... 27.45363004 24.9966625
 27.56763598]


28.366956280965965

In [70]:
xgbm = xgb.XGBRegressor(n_estimators=360, max_depth=6, learning_rate=0.1)
mse_cv(xgbm, train_data[train_cols], train_data['age'])

[16.573492 22.981651 18.655977 ... 29.161343 20.349428 25.83019 ]


27.452074044986922

In [65]:
xgbm = xgb.XGBRegressor(n_estimators=360, max_depth=4, learning_rate=0.1)
xgbm.fit(train_data[train_cols], train_data['age'])
preds_xgbm = xgbm.predict(test_data[train_cols])

In [66]:
preds_xgbm

array([22.898394, 26.787394, 25.222782, ..., 29.536436, 15.682446,
       23.000458], dtype=float32)

In [67]:
solution = pd.DataFrame({"user.id":test_data['user.id'], "age":np.round(preds_xgbm, 0)})
# solution = solution.groupby('user.id', as_index=False)['age'].mean()
solution.to_csv("solutions/word_vecs400_tfidf100_xgbm_rounded.csv", index = False)

In [69]:
train_data.to_csv('train_data_vecs400_tfidf100_grouped.csv', encoding='utf-8')
test_data.to_csv('test_data_vecs400_tfidf100_grouped.csv', encoding='utf-8')

In [117]:
pos_train = pd.read_csv('data/large_files/train_output_pos.csv')

In [118]:
pos_test = pd.read_csv('data/large_files/output_pos.csv')

In [119]:
pos_c = list(pos_train.columns)[-4:]

In [120]:
pos_c = pos_c + ['user.id', 'post.id']

In [121]:
pos_c

['numofdeterminers',
 'numofperspronouns',
 'numofnouns',
 'numofverbs',
 'user.id',
 'post.id']

In [122]:
pos_train = pos_train[pos_c]

In [123]:
pos_c.remove('user.id') 
pos_c.remove('post.id')

In [124]:
pos_c

['numofdeterminers', 'numofperspronouns', 'numofnouns', 'numofverbs']

In [125]:
train_pos = pos_train.groupby('user.id', as_index=False)[pos_c].mean()

In [126]:
pos_c_t = pos_c + ['user.id']
test_pos = pos_test[pos_c_t]

In [129]:
train_data = pd.merge(train_data, train_pos, on='user.id')

In [130]:
test_data = pd.merge(test_data, test_pos, on='user.id')

In [135]:
train_cols = list(train_data.columns)[2:]

In [136]:
train_cols.remove('age')

In [140]:
train_age = pd.read_csv('data/train_age.csv')
test_user = pd.read_csv('data/test_user.csv')

In [142]:
train_counts = train_age.groupby('user.id').size().reset_index(name='counts')

In [144]:
test_counts = test_user.groupby('user.id').size().reset_index(name='counts')

In [146]:
train_data = pd.merge(train_data, train_counts, on='user.id')

In [147]:
test_data = pd.merge(test_data, test_counts, on='user.id')

In [153]:
train_cols = train_cols + ['counts']

In [155]:
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data[train_cols])
krr = KernelRidge(alpha=0.1, kernel='polynomial')
mse_cv(krr, train_scaled, train_data['age'])

[16.48482782 23.56839603 18.8572518  ... 27.43035636 25.09879789
 27.55061108]


28.382981722011966

In [156]:
xgbm = xgb.XGBRegressor(n_estimators=360, max_depth=6, learning_rate=0.1)
mse_cv(xgbm, train_data[train_cols], train_data['age'])

[17.096022 21.913189 18.400118 ... 27.019749 20.379425 27.04165 ]


27.76149518403201

In [162]:
fltr = list(train_data.columns)[:485]

In [164]:
fltr = fltr + ['counts', 'age']

In [166]:
train_data = train_data[fltr]
fltr.remove('age')
test_data = test_data[fltr]

In [170]:
train_cols = list(train_data.columns)[2:]

In [171]:
train_cols.remove('age')

In [172]:
xgbm = xgb.XGBRegressor(n_estimators=360, max_depth=6, learning_rate=0.1)
mse_cv(xgbm, train_data[train_cols], train_data['age'])

[14.566526 22.769083 16.791788 ... 28.430418 21.7513   26.601719]


28.474521233172837

In [285]:
tfidf_train = pd.read_csv('data/large_files/bool_train_tf-idf.csv')

In [286]:
tfidf_test = pd.read_csv('data/large_files/bool_test_tf-idf.csv')

In [287]:
train_data = pd.read_csv('train_data_vecs400_tfidf200_postags_grouped.csv', encoding='utf-8')
test_data = pd.read_csv('test_data_vecs400_tfidf200_postags_grouped.csv', encoding='utf-8')

In [292]:
train_data = train_data[train_cols]
train_cols.remove('age')
test_data = test_data[train_cols]

In [293]:
train_data = pd.merge(train_data, tfidf_train, on='user.id')

In [294]:
test_data = pd.merge(test_data, tfidf_test, on='user.id')

In [290]:
train_cols = list(train_data.columns)[2:488]

In [291]:
train_cols

['user.id',
 'length_text',
 'Positive',
 'Negative',
 'Neutral',
 'Compound',
 'gender_female',
 'gender_male',
 'gender_nan',
 'sign_Aquarius',
 'sign_Aries',
 'sign_Cancer',
 'sign_Capricorn',
 'sign_Gemini',
 'sign_Leo',
 'sign_Libra',
 'sign_Pisces',
 'sign_Sagittarius',
 'sign_Scorpio',
 'sign_Taurus',
 'sign_Virgo',
 'sign_nan',
 'topic_Accounting',
 'topic_Advertising',
 'topic_Agriculture',
 'topic_Architecture',
 'topic_Arts',
 'topic_Automotive',
 'topic_Banking',
 'topic_Biotech',
 'topic_BusinessServices',
 'topic_Chemicals',
 'topic_Communications-Media',
 'topic_Construction',
 'topic_Consulting',
 'topic_Education',
 'topic_Engineering',
 'topic_Environment',
 'topic_Fashion',
 'topic_Government',
 'topic_HumanResources',
 'topic_Internet',
 'topic_InvestmentBanking',
 'topic_Law',
 'topic_LawEnforcement-Security',
 'topic_Manufacturing',
 'topic_Maritime',
 'topic_Marketing',
 'topic_Military',
 'topic_Museums-Libraries',
 'topic_Non-Profit',
 'topic_Publishing',
 'top

In [253]:
# train_data = train_data[train_cols[:485]]
# train_cols.remove('age')
# test_data = test_data[train_cols[:485]]

In [296]:
train_cols = list(train_data.columns)

In [297]:
train_cols.remove('user.id')
train_cols.remove('age')
train_cols.remove('Unnamed: 0')

In [298]:
xgbm = xgb.XGBRegressor(n_estimators=360, max_depth=4, learning_rate=0.1)
mse_cv(xgbm, train_data[train_cols], train_data['age'])

[17.080994 23.111319 17.486294 ... 27.814154 24.761242 25.055666]


27.778521387728716

In [345]:
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data[train_cols])
krr = KernelRidge(alpha=0.001, kernel='polynomial')
mse_cv(krr, train_scaled, train_data['age'])

KeyError: "['Abril' 'Agosto' 'April' 'Aprill' 'August' 'Avgust' 'December' 'Dezember'\n 'Dezembro' 'Februar' 'February' 'Fevereiro' 'Jaanuar' 'Janeiro' 'Januar'\n 'January' 'Julho' 'Juli' 'July' 'June' 'Junho' 'Juni' 'Juuli' 'Juuni'\n 'Mai' 'Maio' 'March' 'May' 'November' 'Novembro' 'October' 'Outubro'\n 'September' 'Setembro' 'abril' 'agosto' 'august' 'augusti' 'augustus'\n 'avril' 'czerwiec' 'desember' 'diciembre' 'elokuu' 'enero' 'febrero'\n 'februarie' 'giugno' 'ianuarie' 'iulie' 'janvier' 'juillet' 'juin' 'juli'\n 'julio' 'juni' 'junio' 'kolovoz' 'lipanj' 'lipiec' 'luglio' 'maart' 'mai'\n 'maj' 'mars' 'marzo' 'mayo' 'mei' 'noiembrie' 'novembre' 'noviembre'\n 'octobre' 'octubre' 'ottobre' 'september' 'septembre' 'septembrie'\n 'septiembre' 'toukokuu'] not in index"

In [187]:
train_data.to_csv('train_data_vecs400_tfidf200_postags_grouped.csv', encoding='utf-8')
test_data.to_csv('test_data_vecs400_tfidf200_postags_grouped.csv', encoding='utf-8')

In [188]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [300]:
user_month_train = pd.DataFrame(columns=['user.id', 'month'])

In [301]:
months = [a[1] for a in train['date'].str.split(',')]

In [302]:
user_id = train['user.id']

In [303]:
user_month_train['user.id'] = user_id
user_month_train['month'] = months

In [304]:
user_month_test = pd.DataFrame(columns=['user.id', 'month'])
months = [a[1] for a in test['date'].str.split(',')]
user_id = test['user.id']
user_month_test['user.id'] = user_id
user_month_test['month'] = months

In [305]:
user_month = pd.concat([user_month_train, user_month_test])

In [306]:
train_month_dummy = pd.get_dummies(user_month['month'])

In [307]:
user_month = pd.concat([user_month, train_month_dummy], axis=1)

In [311]:
month_cols = list(user_month)[3:]

In [313]:
user_month = user_month.groupby('user.id', as_index=False)[month_cols].mean()

In [314]:
train_data_m = pd.merge(train_data, user_month, on="user.id")

In [315]:
test_data_m = pd.merge(test_data, user_month, on="user.id")

In [322]:
train_cols = list(train_data_m.columns)[1:]

In [324]:
train_cols.remove('Unnamed: 0')
train_cols.remove('age')

In [328]:
xgbm = xgb.XGBRegressor(n_estimators=360, max_depth=4, learning_rate=0.1)
mse_cv(xgbm, train_data_m[train_cols], train_data_m['age'])

In [352]:
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data_m[train_cols])
krr = KernelRidge(alpha=0.1, kernel='polynomial')
mse_cv(krr, train_scaled, train_data_m['age'])

[12.10521216 21.38146223 18.21504447 ... 18.89051331 22.11635849
 23.30443903]


30.023120246280765

In [351]:
krr_preds, xgbm_preds, true = get_preds_models_krr(krr, xgbm, train_data_m[train_cols], train_data_m['age'])

In [353]:
min_err = 5
for i in range(50):
    for j in range(50):
        a1 = (1/50)*i
        a2 = (1/50)*j
        preds = a1*krr_preds + a2*xgbm_preds
        err = mean(np.absolute(true - preds))
        if err < min_err:
            print(err)
            min_err = err
            selecteda1 = a1
            selecteda2 = a2

4.819257928690779
4.550717576142901
4.3189104796297215
4.126798957747534
3.973023894926263
3.861124095005923
3.7960376530864663
3.7751203098175976
3.7707903633859807
3.7704954438399607
3.766797517248386
3.758223054792814
3.754348843624292
3.7480356048052443
3.745080413622212
3.74415823629978
3.7417125917551415


In [355]:
selecteda2

0.8200000000000001

In [341]:
xgbm = xgb.XGBRegressor(n_estimators=360, max_depth=4, learning_rate=0.1)
xgbm.fit(train_data_m[train_cols], train_data_m['age'])
preds_xgbm = xgbm.predict(test_data_m[train_cols])

In [343]:
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data_m[train_cols])
test_scaled = scaler.fit_transform(test_data_m[train_cols])

krr = KernelRidge(alpha=0.1, kernel='polynomial')
krr.fit(train_scaled, train_data_m['age'])
preds_krr = krr.predict(test_scaled)

In [358]:
preds = selecteda1*preds_krr + selecteda2*0.8 

In [357]:
solution = pd.DataFrame({"user.id":test_data['user.id'], "age":preds})
# solution = solution.groupby('user.id', as_index=False)['age'].mean()
solution.to_csv("solutions/word_vecs400_booltfidf200_xgbm.csv", index = False)